In [1]:
import os
import io
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array      # New import
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from ipywidgets import FileUpload, Image, VBox
from IPython.display import display

# Enable mixed precision training for speed boost


dataset_path = "Training"
img_width, img_height = 150, 150  # Reduced size to 160x160 for speed improvement
batch_size = 32

In [2]:
from PIL import Image

# Walk through all subdirectories in the dataset path
for subdir, dirs, files in os.walk(dataset_path):
    for file in files:
        img_path = os.path.join(subdir, file)
        if os.path.isfile(img_path) and os.path.splitext(img_path)[1].lower() == '.png':
            im = Image.open(img_path)
            # Check if the image is in palette mode or not RGBA
            if im.mode in ('P', 'LA') or (im.mode == 'RGBA' and 'transparency' in im.info):
                im = im.convert("RGBA")
                im.save(img_path)  # Overwrite the original image


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,       
    width_shift_range=0.2,   
    height_shift_range=0.2,  
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,  

    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(  

    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 67000 images belonging to 1025 classes.
Found 16241 images belonging to 1025 classes.


In [4]:
# ---- Model Architecture ----
model = Sequential([
    Conv2D(32, (3, 3), activation=LeakyReLU(alpha=0.1), input_shape=(img_width, img_height, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation=LeakyReLU(alpha=0.1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation=LeakyReLU(alpha=0.1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation=LeakyReLU(alpha=0.1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(512, (3, 3), activation=LeakyReLU(alpha=0.1)), # Increased number of filters
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation=LeakyReLU(alpha=0.1)),  # Added another dense layer
    BatchNormalization(),
    Dropout(0.5),             
    Dense(1025, activation='softmax') 
])

# ---- Model Compilation and Training ----
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)  

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)  



In [5]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,             
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,  

    callbacks=[reduce_lr, early_stopping]
)

Epoch 1/50
2093/2093 [==============================] - 908s 431ms/step - loss: 6.4984 - accuracy: 0.0131 - val_loss: 6.0966 - val_accuracy: 0.0216 - lr: 0.0010
Epoch 2/50
2093/2093 [==============================] - 578s 276ms/step - loss: 5.4887 - accuracy: 0.0499 - val_loss: 5.5154 - val_accuracy: 0.0547 - lr: 0.0010
Epoch 3/50
2093/2093 [==============================] - 478s 229ms/step - loss: 4.9320 - accuracy: 0.0973 - val_loss: 4.9168 - val_accuracy: 0.1141 - lr: 0.0010
Epoch 4/50
2093/2093 [==============================] - 474s 227ms/step - loss: 4.4771 - accuracy: 0.1510 - val_loss: 4.6193 - val_accuracy: 0.1521 - lr: 0.0010
Epoch 5/50
2093/2093 [==============================] - 476s 227ms/step - loss: 4.1228 - accuracy: 0.1991 - val_loss: 4.3069 - val_accuracy: 0.2045 - lr: 0.0010
Epoch 6/50
2093/2093 [==============================] - 480s 230ms/step - loss: 3.8422 - accuracy: 0.2411 - val_loss: 4.2219 - val_accuracy: 0.2210 - lr: 0.0010
Epoch 7/50
2093/2093 [============

In [6]:
model.save("pokemon.keras")

In [20]:
import os
import io
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from ipywidgets import FileUpload, Image, VBox
from IPython.display import display

# Load the saved model
model = load_model("pokemon.keras")

# Path to the directory containing the Pokémon images
dataset_path = "Training"

# Define image dimensions
img_width, img_height = 150, 150

# Function to preprocess and resize the image for prediction
def preprocess_image(img_array):
    img_array = tf.image.resize(img_array, [img_width, img_height])
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize the image
    return img_array
    
# Function to handle the uploaded image and display predictions
def on_upload_change(change):
    uploaded_file = change['new']
    if uploaded_file:
        content = uploaded_file[0]['content']
        img = load_img(io.BytesIO(content))
        img_array = img_to_array(img)

        # Preprocess and predict the results
        img_array = preprocess_image(img_array)
        prediction = model.predict(img_array)
        predicted_class = np.argmax(prediction)
        
        # Get the label (Pokémon name) and similarity percentage of the predicted class
        predicted_pokemon = train_generator.class_indices
        predicted_pokemon = dict((v, k) for k, v in predicted_pokemon.items())
        predicted_pokemon = predicted_pokemon[predicted_class]
        similarity_percentage = round(np.max(prediction[0]) * 100, 2)  # Get percentage and round to 2 decimal places

        # Load the image of the predicted Pokémon class
        predicted_image_path = os.path.join(dataset_path, predicted_pokemon, os.listdir(os.path.join(dataset_path, predicted_pokemon))[0])
        predicted_img = load_img(predicted_image_path, target_size=(img_width, img_height))

        # Plot the images
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
        axes[0].imshow(img)
        axes[0].set_title("Uploaded Image")
        axes[0].axis('off')
        axes[1].imshow(predicted_img)
        axes[1].set_title(f"Predicted Pokémon: {predicted_pokemon} ({similarity_percentage}%)")
        axes[1].axis('off')
        plt.show()

# Set up the file upload button
upload_button = FileUpload(accept='image/*', multiple=False)
upload_button.observe(on_upload_change, names='value')

display(VBox([upload_button]))
